In [65]:
library(caret)
library(LaplacesDemon)
library(splines2)
library(matrixcalc)
library(pracma)
library(expm)
library(doParallel)
library(foreach)
library(plyr)
library(dplyr)
library(Matrix)
library(MASS)
library(fastICA)

In [66]:
make_rho_mat <- function(rho,p){
  
  the_vec <- matrix(NA, nrow = p, ncol =  p)
  
  for (i in 1:(p)){
    
    for (j in 1:p){
      
      the_vec[i,j] <- rho^abs(i-j)
      
    }
  }
  
  return(.5*(the_vec+t(the_vec)))
  
}


qarb <- function(emp_dist){
  
  quantile_func <- function(p){
  
    initial_dist <- quantile(emp_dist, p, na.rm=TRUE)
    
    initial_dist[is.infinite(initial_dist)] <- 0
    
    return(initial_dist)
  
  }
  
  return(quantile_func)
  
}

norta <- function(n_obs, corr_mat, distribution=qnorm){
  
  Zs <- rmvn(n=n_obs, mu=rep(0, dim(corr_mat)[1]), Sigma=corr_mat)
  
  phi_Zs <- pnorm(Zs)
  
  phi_Zs[phi_Zs %in% c(0,1)] <- .5
  
  desired_variables <- apply(phi_Zs, MARGIN=1, FUN=distribution)
   
  return(desired_variables)
  
}

matrixToPower <- function(aMatrix, power){ ### Utility for calculating matrices to given powers
                                           ### Assumes square, symmetric matrix
  matrixEigenDecomp <- eigen(aMatrix)
  
  matrixPowered <- matrixEigenDecomp$vectors %*% diag(matrixEigenDecomp$values^power) %*% t(matrixEigenDecomp$vectors)
  
  return(matrixPowered)
  
}



ICAmodelDataMaker <- function(n, p, quantile_func, rho=.5, rho_misspecification=0, rho_gamma=.2){

  misspecificationMat <- make_rho_mat(rho_misspecification, p)
  
  #XUnmixed <- matrixToPower(misspecificationMat, -.5) %*% XUnmixedPure
  
  ### True U in so(p)
  UTrue <- gramSchmidt(array(rnorm(p^2), dim=rep(p, 2)))$Q 
  
  ### True covariance matrix for data
  covMat <- make_rho_mat(rho, dim(UTrue)[1])
  
  Sigma_g <- make_rho_mat(rho_gamma, dim(UTrue)[1])
  
  ### Always the identity matrix for these simulations
  sigmaTilde <- Sigma_g %*% Sigma_g
  
  ### Covariance matrix to the -1/2 power
  covMatMinusHalf <- matrixToPower(covMat, -.5)
  
  ### Sigma tilde to the -1/2 power
  sigmaTildeMinusHalf <- matrixToPower(sigmaTilde, -.5)
    
  ### True unmixing matrix
  trueW <- UTrue %*% covMatMinusHalf
  
  ### True unmixing matrix for data with covariance Sigma tilde
  trueWTilde <- UTrue %*% sigmaTildeMinusHalf
    
  ### Generate unmixed data with NORTA
  XUnmixed <- norta(n, corr_mat = misspecificationMat, distribution = quantile_func)
  
  XUnmixed <- matrixToPower(cov(t(XUnmixed)), -.5)  %*% XUnmixed
    
  ### The observed, mixed data
  X <- solve(trueW) %*% XUnmixed 
  
  oneVec <- matrix(1, nrow=dim(UTrue)[2], ncol=1)
  
  ### The true constants needed for estimating the score function
  cTrueWTilde <- apply(trueWTilde , MARGIN=1, 
                       FUN = function(x) ( t(x) %*% sigmaTilde %*% t(t(x))))
  
  finalOutput <- list(XUnmixed=XUnmixed, trueW=trueW, trueWTilde=trueWTilde, 
                      sigmaTilde=sigmaTilde, X=X, cTrueWTilde=cTrueWTilde)

}

In [67]:
require( Rcpp )

#  Source code for our function
func <- 'NumericMatrix mmult( NumericMatrix m , NumericVector v , bool byrow = true ){
  if( byrow );
    if( ! m.nrow() == v.size() ) stop("Non-conformable arrays") ;
  if( ! byrow );
    if( ! m.ncol() == v.size() ) stop("Non-conformable arrays") ;

  NumericMatrix out(m) ;

  if( byrow ){
    for (int j = 0; j < m.ncol(); j++) {
      for (int i = 0; i < m.nrow(); i++) {
        out(i,j) = m(i,j) * v[j];
      }
    }
  }
  if( ! byrow ){
    for (int i = 0; i < m.nrow(); i++) {
      for (int j = 0; j < m.ncol(); j++) {
        out(i,j) = m(i,j) * v[i];
      }
    }
  }
  return out ;
}'

#  Make it available
cppFunction( func )


In [109]:
scoreFunctionUpdate <- function(Z, W, SigmaTilde, lambdaGrid=10^seq(-6, 6, 1), 
                                numKnots=10, gridStep=.01,
                                    nFolds=5){
    
  options(warn=-1)
    
    
  basisEvalInLongVec <- function(basisList, zVec){
    
    
    basisEval <- unlist(lapply(1:length(basisList), 
                               FUN=function(x) predict(basisList[[x]], zVec[x])))
                               
    return(array(basisEval, dim=c(length(basisEval), 1)))
    
    }
                               
                               
  evalInBlockDiag <- function(basisList, zVec){
    
    basisEval <- lapply(1:length(basisList), 
                        FUN=function(x) predict(basisList[[x]], zVec[x]))
                               
    return(as.array(bdiag(basisEval)))
    
    }
                        
                        
  evalAsList <- function(basisList, zVec){
    
    basisEval <- lapply(1:length(basisList), 
                        FUN=function(x) predict(basisList[[x]], zVec[x]))
                               
    return(basisEval)
    
    }
                        
                        
  knotMaker <- function(z, numKnots=10){
    
      num_knots_plus_1 <- numKnots+1 ### Give the number of knots, plus 1
  
      theProbs <- seq(1:(num_knots_plus_1-1))/(num_knots_plus_1)
  
      theKnots <- quantile(Z, probs = theProbs) ### Knots based on quantiles
  
  ### Making sure knots are not placed are extreme points in the data
      if (min(theKnots) == min(z)){
    
        theKnots[which(theKnots==min(z))] <- min(z[which(z != min(z))])
    
    
      }
  
      if (max(theKnots) == max(z)){
    
        theKnots[which(theKnots==max(z))] <- min(Z[which(z != max(z))])
    
    
      }
  
  ### Generating the basis to be used to represent the score function
      theKnots <- unique(theKnots)
    
      return(theKnots)
    
    }


    gridMaker <- function(z, gridStep=.001){
    
      zMin <- floor(min(z, na.rm = T))
  
      zMax <- ceiling(max(z, na.rm = T))
  
      zGrid <- seq(zMin, zMax,  gridStep)
    
    }
  
  ### Calculates the risk
  riskCalculator <- function(betaHat, lambda, T1, T2, cW){
    
    riskValue <- t(betaHat) %*% T1 %*% betaHat + 2*t(cW) %*% T2 %*% betaHat
    
    return(riskValue)
    
  }
  
  ### Calculates the basis coefficient vector for the score function
  betaCalculator <- function(lambda, T1, T2, T3, cW){
          
    betaHat <- -1*ginv(T1+lambda*T3) %*% t(T2) %*% cW
      
    
    return(betaHat)
    
  }
  
  ### If bigger than 1, perform cross validation, otherwise estimate from given turning parameter
  CV <- length(lambdaGrid) > 1
    
    
  if (CV & (nFolds==1)){
      
      stop("Must have multiple folds if doing CV.")
      
  }
  
  ### Number of curves
  n <- dim(Z)[2]

  K <- dim(Z)[1]
  
  knots <- alply(Z, 1, knotMaker, numKnots=10)

  grids <- alply(Z, 1, gridMaker)
    
    #### number of knots = df-degree
   #### Specify knots to be at quantiles 
  
  df <- numKnots+1+min(3,n)
                        
  bases<- lapply(1:K,FUN = function(x) bSpline(grids[[x]], degree = min(3,n), df=df,
                                               intercept = T)) ### use knots
                 
                 
  WSigmaTildeTW <- W %*% SigmaTilde %*% t(W)
      
  oneVec <- array(rep(1, dim(W)[1]), dim=c(dim(W)[1], 1))  
      
  cW <- array(apply(W, MARGIN=1, FUN=function(x) t(oneVec) %*% ( SigmaTilde * x %*% t(x) ) %*% oneVec), dim=c(length(oneVec), 1))
      
  n <- dim(Z)[2]
      
  WSigmaTildeTWBigChungus <-  apply(t(apply(WSigmaTildeTW, MARGIN=1, 
                                              FUN=rep, each=dim(bases[[1]])[2])), 
                                      MARGIN=2, FUN=rep, each=dim(bases[[1]])[2])
    
  allTheEvals <- alply(Z, 2, basisEvalInLongVec, basisList=bases)
      
  allTheCrossProds <- lapply(allTheEvals, FUN=function(x) crossprod(t(x)))  
      
  basesDerivs <- lapply(bases, FUN=deriv, derivs=1L)
      
  derivEvals <- alply(Z, 2, evalInBlockDiag, basisList=basesDerivs)
                             
  basesSecondDerivs <- lapply(bases, FUN=deriv, derivs=2L)
                             
  T1 <- WSigmaTildeTWBigChungus*Reduce('+', allTheCrossProds)/length(allTheCrossProds)
      
  T2 <- Reduce("+", derivEvals)/length(derivEvals) 
    
  T3 <- as.matrix(bdiag(lapply(basesSecondDerivs, FUN=function(x) gridStep*base::crossprod(x) )))
  
  if (CV){ ### Cross-validation for tuning parameter estimation
    
    cvFolds <- createFolds(1:n, k =nFolds)
    
    ### Cross validation done in parallel
    cvErrorsForLambdas <- foreach(lamb=lambdaGrid,
                .packages=c('splines2', "caret", 'MASS', 'plyr', 'Matrix'), .combine = 'c') %dopar% {
      
      risksThisLambda <- c()
      
      cvFolds <- createFolds(1:n, k =nFolds)
      
      for (fold in cvFolds){
      
        trainZ <- Z[, -fold]
        
        valZ <- Z[, fold]
          
        T1Train <- WSigmaTildeTWBigChungus*Reduce('+', allTheCrossProds[-fold])/length(allTheCrossProds[-fold])
          
        T1Val <- WSigmaTildeTWBigChungus*Reduce('+', allTheCrossProds[fold])/length(allTheCrossProds[fold])
      
        T2Train <- Reduce("+", derivEvals[-fold])/length(derivEvals[-fold]) 
          
        T2Val <- Reduce("+", derivEvals[fold])/length(derivEvals[fold]) 
    
        T3Train <- as.matrix(bdiag(lapply(basesSecondDerivs, FUN=function(x) gridStep*base::crossprod(x) )))
      
        betaHatThisFoldThisLambda <- betaCalculator(lambda=lamb, T1=T1Train, T2=T2Train, T3=T3, cW=cW)
        
        riskThisFoldThisLambda <- riskCalculator(betaHat=betaHatThisFoldThisLambda, 
                                                 lambda=lamb, T1=T1Val, T2=T2Val, cW=cW)
        
        risksThisLambda <- c(risksThisLambda, riskThisFoldThisLambda)
      
      }
     
      mean(risksThisLambda)
       
    }
    
    chosenLambda <- lambdaGrid[which.min(cvErrorsForLambdas)]
    
  }else{
    
    chosenLambda <- lambdaGrid
    
  }
  
  bigBetaHat <- betaCalculator(lambda=chosenLambda, T1=T1, T2=T2, T3=T3, cW=cW)
                                          
  basesDimensionSizes <- unlist(lapply(bases, FUN=function(x) dim(x)[2]))
                                
  betaBreakPoints <- cumsum(basesDimensionSizes)-basesDimensionSizes+1
                                          
  brokenBeta <- Map(function(i,j) bigBetaHat[i:j, , drop=FALSE], 
                    betaBreakPoints, cumsum(diff(c(betaBreakPoints, dim(bigBetaHat)[1]+1))))
                                          
  scoreFunctionCoordinate <- function(basis, betaHat){
      
    finalCoordinate <- function(z, derivativeOrder=0){ ### Note, z is a scalar!
        
        if (derivativeOrder==0){

          basisNow = basis

        }else{

        basisNow <- deriv(basis, derivs = derivativeOrder)
    
    }
    
    basisVec = predict(basisNow, newx = z)
    
    return(basisVec %*% betaHat)
        
        }
      
      return(finalCoordinate)
    
  }
            
  #force(scoreFunctionCoordinate)
                                       
  scoreFunctionList <- lapply(1:length(brokenBeta), FUN = function(k)
      scoreFunctionCoordinate(basis=bases[[k]], betaHat=brokenBeta[[k]])  )      
                              
  scoreListToVector <- function(aScoreList){
    
    ### The output. Its arguments are a numberic vector, and an argument for desired order of derivative
    scoreVector <- function(theta, derivativeOrder=0){
      
      if (!is.matrix(theta)){
        
        theta <- array(theta, dim=c(length(theta), 1))
        
      }
      
      return(t(sapply(1:dim(theta)[1], FUN=function(x) aScoreList[[x]](theta[x,], derivativeOrder=derivativeOrder))))
      
    }
    
    return(scoreVector)
    
    
  }
               
  finalScoreVector <- scoreListToVector(aScoreList = scoreFunctionList)
  
  return(finalScoreVector)
  
}

In [3]:
desired_clusters <- 4

cl <- makeCluster(desired_clusters)

registerDoParallel(cl)

In [69]:
newThing <- rgamma(10000, shape=1, scale=4)

theQuantileFunction <- qarb(newThing)

theGoods <- ICAmodelDataMaker(n=2500, p=4, quantile_func=theQuantileFunction, rho=.5, rho_misspecification=0, rho_gamma=.2)

In [70]:


newThing <- rgamma(10000, shape=1, scale=4)

theQuantileFunction <- qarb(newThing)

theGoods <- ICAmodelDataMaker(n=2500, p=4, quantile_func=theQuantileFunction, rho=.5, rho_misspecification=0, rho_gamma=.2)


X <- theGoods$trueW %*% theGoods$XUnmixed

covTheta <- cov(t(X)) ### Estimated covariance of data
  
  ### Covariance of data to the -1/2 power
  covThetaMinusHalf <- matrixToPower(covTheta, -.5)

   fastICAInfo <- fastICA(t(X), n.comp = dim(theGoods$XUnmixed)[1])
    
   WEst <- t(fastICAInfo$K %*% fastICAInfo$W)
    
   UWarmStart <- WEst%*% solve(covThetaMinusHalf)


WEst<- UWarmStart %*% covThetaMinusHalf

Z <- WEst %*% X
 ### need to add unmixing matrix to argument
SigmaTilde <- theGoods$sigmaTilde

lambdaGrid=10^seq(-6, 6, 1)
numKnots=10
gridStep=.01
nFolds=5

In [71]:
options(warn=-1)

In [110]:
startTime <- Sys.time()

aThing <- scoreFunctionUpdate(Z=Z, W=WEst, SigmaTilde=SigmaTilde, lambdaGrid=10^seq(-6, 6, 1), numKnots=10, gridStep=.01,
                                    nFolds=5)

endTime <- Sys.time()


print(difftime(endTime, startTime))

Time difference of 7.739304 secs


In [121]:
thetaStarMatrix <- X
UInit <- UWarmStart

sigmaTilde <- SigmaTilde

scoreFunctionVectorInit <- aThing
minIterationsU=1
maxIterationsU=2
minChangeU=1e-06

In [122]:
  Gamma <- cov(t(thetaStarMatrix))
    
  thetaTildeMatrix <- matrixToPower(Gamma, -.5) %*% thetaStarMatrix
  
  ### Note that A is a symmetric matrix
  A <- matrixToPower(Gamma, -.5) %*% sigmaTilde %*% matrixToPower(Gamma, -.5)
    
  ### The value of the risk
  rHat <- function(uRisk, ourScoreVector, thetaTildeMatrix, A){ 
      
    UtAUrisk <- uRisk %*% A %*% t(uRisk)
    
    uncoupledThetaTildesUrisk <- uRisk %*% thetaTildeMatrix
      
    riskScoreVals <- ourScoreVector( uncoupledThetaTildesUrisk, derivativeOrder=0)
      
    riskScoreValsPrime <- ourScoreVector( uncoupledThetaTildesUrisk, derivativeOrder=1)
    
    ## This should be trace
    firstPart <- tr(t(riskScoreVals) %*% UtAUrisk %*% riskScoreVals)/dim(riskScoreVals)[2]
      
    secondPart <- mean(2*array(diag(UtAUrisk), dim=c(1, dim(riskScoreValsPrime)[1])) %*% riskScoreValsPrime)
    
    return(firstPart+secondPart)
    
  }
  
  ### The gradient of the risk
  nablaRHat <- function(uRisk, thetaTildeMatrix, scoreFunctionVector, A){      
      
    alternateCrossTermCalculatorForThetaTilde <- function(thetaMat, scoreVectorMat, 
                                                          scoreVectorPrimeMat, UtAU){
        
        finalTermMatrix <- array(NA, dim=rep(dim(scoreVectorMat)[1], 2))
        
        IkByk <- diag(rep(1, dim(scoreVectorMat)[1]))
        
        for (l in 1:dim(thetaTildeMatrix)[1]){
        
            e_l <- array(rep(0, dim(scoreVectorMat)[1]), dim=c(dim(scoreVectorMat)[1], 1) )

            e_l[l,1] <- 1

            neededPrimeVec <- scoreVectorPrimeMat[l,]

            G_l <- scoreVectorMat * sapply(neededPrimeVec, FUN=rep, dim(scoreVectorMat)[1])

            finalTermMatrix[l,] <- (2/dim(scoreVectorMat)[2])*(UtAU[l,] %*% (IkByk-(e_l %*% t(e_l)))
                                                              %*% G_l %*% t(thetaMat)) 
            
            }
        
        return(finalTermMatrix)

    }
      
    uncoupledThetaTildesUrisk <- uRisk %*% thetaTildeMatrix
      
    scoreVals <- scoreFunctionVector(uncoupledThetaTildesUrisk, derivativeOrder=0)
      
    scoreValsPrime <- scoreFunctionVector(uncoupledThetaTildesUrisk, derivativeOrder=1)
      
    scoreValsDoublePrime <- scoreFunctionVector(uncoupledThetaTildesUrisk, derivativeOrder=2)
     
    UtAU <- uRisk %*% A %*% t(uRisk)
      
      
    ### Terms that get multiplied to U, no cross terms
    UkTermsNonMatrix <- rowMeans(2*(scoreVals^2)+4*scoreValsPrime)
      
    UkTermsMatrix <- t(sapply(UkTermsNonMatrix, FUN=rep, dim(uRisk)[1]))
      
    firstPart <- UkTermsMatrix * (uRisk %*% t(A)) ### Need row means
      
    ### Terms that get multiplied to theta tilde  
    thetaTildeTermsMatrix <- t(sapply(diag(UtAU), 
                                      FUN=rep, 
                                      dim(scoreVals)[2]))*(2*scoreVals*scoreValsPrime+
                                                                    2*scoreValsDoublePrime)  
      
    secondPart <- (thetaTildeTermsMatrix %*% t(thetaTildeMatrix))/dim(thetaTildeMatrix)[2]
      
      
    ### Terms that get multiplied to U, with cross terms  
      
      
    thirdPartU <- (1/dim(thetaTildeTermsMatrix)[2]) * (scoreVals %*% t(scoreVals)-diag(diag(scoreVals %*% t(scoreVals)))) %*% (uRisk %*% t(A))

      
    thirdPartThetaCrossTerms <-  alternateCrossTermCalculatorForThetaTilde(thetaMat=thetaTildeMatrix,
                                 scoreVectorMat=scoreVals, scoreVectorPrimeMat=scoreValsPrime, 
                                  UtAU=UtAU)
      
    nablaValues <- firstPart + secondPart + thirdPartU + thirdPartThetaCrossTerms

    return(nablaValues)
    
  } 
  
  scoreFunctionVector <- scoreFunctionVectorInit
  
  UtMinus1 <- UInit
  
  dist <- 1
  
  alpha <- .2
  
  etaGrid <-10*c(.5^seq(0, 15, 1), 0)
  
  thingy <- 1
  
  for (iteratorU in 1:maxIterationsU){
    
    ### Unmix decorrelated data
    uncoupledThetaTildes <- UtMinus1 %*% thetaTildeMatrix 
    
    ### Gradient of risk
    nablaRHat_t <- nablaRHat(uRisk=UtMinus1, thetaTildeMatrix=thetaTildeMatrix, scoreFunctionVector,A=A)
    
    ### Geodesic gradient
    nablaRHatTilde <- nablaRHat_t %*% t(UtMinus1) - UtMinus1 %*% t(nablaRHat_t)
    
    Ht <- nablaRHatTilde
    
    ### We normalize the gradient as described in the notes
    Ht <- sqrt(2)*Ht/norm(Ht, "F")
    
    expEtaHs <- sapply(etaGrid, FUN=function(x) expm::expm(-1*x * Ht, order=16), simplify = F)
    
    potentialUs <- lapply(expEtaHs, FUN=function(x) x %*% UtMinus1)
    
    riskSatisfactionTerms <- function(x, y){

      return(c(rHat(uRisk=x, ourScoreVector=scoreFunctionVector, thetaTildeMatrix=thetaTildeMatrix, A=A), 
               rHat(uRisk=UtMinus1, ourScoreVector=scoreFunctionVector,thetaTildeMatrix=thetaTildeMatrix, A=A)- 
               alpha*.5*y* (norm(Ht, "F")^2)))
      
    }
    
    neededRiskTerms <- mapply(riskSatisfactionTerms, x=potentialUs, y=etaGrid)
    
    doesItSatisfy <- apply(neededRiskTerms, 2, FUN = function(x) x[1] >= x[2])
                           
    doesItSatisfy[is.na(doesItSatisfy)] <- TRUE
    
    
    if(all(doesItSatisfy)){
      ### If we can't improve, we choose our step to be the identity matrix (so no change)
      expEtaH <- expEtaHs[[length(expEtaHs)]]
      
    }else{
      ### Choose the risk minimizer 
      expEtaH <- expEtaHs[[which.min(neededRiskTerms[1,])]]
    
    }
    ### Update U
    Ut <- expEtaH %*% UtMinus1
    
    ### Percentage change in risk
    dist <- (rHat(uRisk=UtMinus1, ourScoreVector=scoreFunctionVector,thetaTildeMatrix=thetaTildeMatrix,A=A)-
                   rHat(uRisk=Ut, ourScoreVector=scoreFunctionVector,  
                thetaTildeMatrix, A=A))/abs(rHat(uRisk=UtMinus1, ourScoreVector=scoreFunctionVector,
                                            thetaTildeMatrix=thetaTildeMatrix, A=A))

    overMinIterationsU <- iteratorU > minIterationsU
    
    overMaxIterationsU <- iteratorU > maxIterationsU
    
    closerThanDistU <- dist < minChangeU
      
    if (overMinIterationsU & closerThanDistU){
        
      break
        
    }else{ 
        
      UtMinus1 <- Ut
        
      }
      
  }

In [74]:
uncoupledThetaTildes <- UtMinus1 %*% thetaTildeMatrix 

In [75]:
### Gradient of risk
    nablaRHat_t <- nablaRHat(uRisk=UtMinus1, thetaTildeMatrix=thetaTildeMatrix, scoreFunctionVector,A=A)

In [76]:
startTime <- Sys.time()

nablaRHat_t <- nablaRHat(uRisk=UtMinus1, thetaTildeMatrix=thetaTildeMatrix, scoreFunctionVector,A=A)

endTime <- Sys.time()


print(difftime(endTime, startTime))

Time difference of 13.99946 secs


In [77]:
startTime <- Sys.time()

nablaRHatTilde <- nablaRHat_t %*% t(UtMinus1) - UtMinus1 %*% t(nablaRHat_t)
    
    Ht <- nablaRHatTilde
    
    ### We normalize the gradient as described in the notes
    Ht <- sqrt(2)*Ht/norm(Ht, "F")
                           
endTime <- Sys.time()


print(difftime(endTime, startTime))
                           

Time difference of 0.002921343 secs


In [78]:
startTime <- Sys.time()

nablaRHatTilde <- nablaRHat_t %*% t(UtMinus1) - UtMinus1 %*% t(nablaRHat_t)
    
    expEtaHs <- sapply(etaGrid, FUN=function(x) expm::expm(-1*x * Ht, order=16), simplify = F)
    
    potentialUs <- lapply(expEtaHs, FUN=function(x) x %*% UtMinus1)
    
    riskSatisfactionTerms <- function(x, y){

      return(c(rHat(uRisk=x, ourScoreVector=scoreFunctionVector, thetaTildeMatrix=thetaTildeMatrix, A=A), 
               rHat(uRisk=UtMinus1, ourScoreVector=scoreFunctionVector,thetaTildeMatrix=thetaTildeMatrix, A=A)- 
               alpha*.5*y* (norm(Ht, "F")^2)))
      
    }
    
    neededRiskTerms <- mapply(riskSatisfactionTerms, x=potentialUs, y=etaGrid)
    
   # doesItSatisfy <- apply(neededRiskTerms, 2, FUN = function(x) x[1] >= x[2])
                           
endTime <- Sys.time()


print(difftime(endTime, startTime))

Time difference of 4.731042 mins


In [304]:
stopCluster(cl)

In [37]:
require( Rcpp )

#  Source code for our function
func <- 'NumericMatrix mmult( NumericMatrix m , NumericVector v , bool byrow = true ){
  if( byrow );
    if( ! m.nrow() == v.size() ) stop("Non-conformable arrays") ;
  if( ! byrow );
    if( ! m.ncol() == v.size() ) stop("Non-conformable arrays") ;

  NumericMatrix out(m) ;

  if( byrow ){
    for (int j = 0; j < m.ncol(); j++) {
      for (int i = 0; i < m.nrow(); i++) {
        out(i,j) = m(i,j) * v[j];
      }
    }
  }
  if( ! byrow ){
    for (int i = 0; i < m.nrow(); i++) {
      for (int j = 0; j < m.ncol(); j++) {
        out(i,j) = m(i,j) * v[i];
      }
    }
  }
  return out ;
}'

#  Make it available
cppFunction( func )

#  Use it
res1 <- mmult( m , v )

Loading required package: Rcpp



ERROR: Error in mmult(m, v): object 'm' not found
